# 9.1
## T1. [Select Cells in Grid With Maximum Score](https://leetcode.com/problems/select-cells-in-grid-with-maximum-score/description/)(LC)
- 题意: 给定一个矩阵，从每行中选择一个数，要求选择的数不能相等，求最后选出来的数的最大值
- 题解
  思考状态压缩dp。令$dp[i]$表示每一行选择状态为$i$时的最大值，关键在于如何考虑不重复计算每个数值，这个可以通过**枚举每个值**做到，然后枚举该值所在行

In [ ]:
class Solution:
    def maxScore(self, grid: List[List[int]]) -> int:
        n, m = len(grid), len(grid[0])
        pos = defaultdict(list)

        for i, ls in enumerate(grid):
            for v in ls:
                pos[v].append(i)

        dp = [0] * (1 << n)
        for x, ids in pos.items(): 
            ndp = dp[:]
            for i in ids: 
                for mask in range(1 << n):
                    if (mask >> i & 1) == 0:
                        nmask = mask | 1 << i 
                        ndp[nmask] = max(ndp[nmask], dp[mask] + x)
            dp = ndp[:] 
                
        return max(dp)

# 9.2
## T1. [Very Interesting Game](https://codeforces.com/problemset/problem/117/B)
- 题意: 给定三个数a, b, mod, A任意选择一个不大于a的数x，B选择一个不大于b的数y，B要尽量使$x \dot 10^9 + y$可以被mod整除，判断最后谁能赢
- 题解(固定变量)
  让A选择的x固定，此时如何判断B是否可以获胜？
  
  可以将公式扩展为如下形式: $(x \cdot 10^9) \% mod + y \% mod = 0$ i.e., $y \% mod = (-x \cdot 10^9) \% mod$.
  因此只需要判断$(-x \dot 10^9) \% mod$是否大于b即可


In [ ]:
# (x * 10 ^ 9 + y) % mod = 0 
# 判断最小的y是否小于b即可 
# 从小到大枚举x, 判断(-x * 10 ^ 9) % mod > b

a, b, mod = map(int, input().split()) 

ans = -1

for i in range(min(a, mod - 1) + 1):
    if (-i * 10 ** 9) % mod > b: 
        ans = i 
        break 
if ans != -1:
    print(1, str(ans).zfill(9))
else:
    print(2)

## T2. [Lucky Days](https://codeforces.com/problemset/problem/1055/C)
- 题意: 给定两个无限循环的区间，求两个区间的最大交集
- 题解: 两个区间每一步可以移动的距离为$t=gcd(ta, tb)$，只需要考虑两个区间的相对位置即可，考虑相对间距为$d=(la - lb) \% t$。
  
  - 若a在b左边，此时$d>0$，最终答案为$\min \{b - d, a\}$
  - 若a在b的右边，此时$d < 0$，最终答案为$\min \{a - d, b\}$，注意，由于取模后总为正，此时最终答案为$\min \{a + d - t, b\}$

In [ ]:
from math import gcd 

l0, r0, t0 = map(int, input().split()) 
l1, r1, t1 = map(int, input().split()) 

a, b = r0 - l0 + 1, r1 - l1 + 1

t = gcd(t0, t1)
d = (l0 - l1) % t # 相对位置

# 1在0的左边, i.e., d > 0: min(a + d - t, b)
# 1在0的右边, i.e., d < 0: min(b - d, a) 

print(max(0, min(a + d - t, b), min(b - d, a)))

# 9.3
## T1. [Merge Sort](https://codeforces.com/problemset/problem/873/D)
- 题意: 构造一个列表，使得需要且只需要k次扫描，就可以使该列表有序
- 题解: 可以发现，让一个无序的区间变得有序，至少需要两次扫描操作，因此，每次让一个区间变得有序至少会带来2次扫描，而最开始会扫描整个数据，以确定是否有序，k最终必然为奇数；在构造时，只需要模拟整个过程，当区间有序时，将mid和mid-1所在位置进行调换即可

In [ ]:
n, k = map(int, input().split()) 

if k & 1 == 0: 
    print(-1)
    exit() 
k -= 1
ans = list(range(1, n + 1))

def dfs(l, r): 
    global k 
    if k == 0 or l + 1 == r: 
        return 
    k -= 2
    mid = l + r >> 1 
    ans[mid], ans[mid - 1] = ans[mid - 1], ans[mid]
    dfs(l, mid)
    dfs(mid, r)

dfs(0, n)

print(*ans if k == 0 else [-1])

## T2. [Minimum path](https://codeforces.com/problemset/problem/1031/D)
- 题意: 给定一个$n\times n$的矩阵，初始时在左上角，终点在右下，经过的路径会形成一个字符串，给定k次操作，任意修改其中的一个字母，求所能形成的字典序最小的字符串
- 题解:
  贪心: 假设形成了字符串s, k次操作全部给到s的前k个非a字母，将其变成a，这样形成的字符串一定是字典序最小的。

  同时可以发现，在矩阵中找到m个这样的点，从起点到这个点的路径中的非a字母恰好<=k，因此，只需要枚举这些起点，找到其到终点字典序最小的字符串即可 

In [ ]:
# submission url: https://codeforces.com/contest/1031/submission/279405631
# 贪心: 前k个字符全为'a'
# 从初始位置对于不是'a'的字符经过k次操作变成'a'后能达到的最远位置，并从这些位置出发，取路径最小值
n, k = map(int, input().split()) 

a = [input() for _ in range(n)]

fins = [(0, 0)]

cnt = [[n ** 2] * n for _ in range(n)]

cnt[0][0] = a[0][0] != 'a'
for i in range(n):
    for j in range(n): 
        if i + 1 < n: 
            cnt[i + 1][j] = min(cnt[i + 1][j], (a[i + 1][j] != 'a') + cnt[i][j])
        if j + 1 < n:
            cnt[i][j + 1] = min(cnt[i][j + 1], (a[i][j + 1] != 'a') + cnt[i][j])

for i in range(n):
    for j in range(n):
        if cnt[i][j] == k:
            if i + 1 < n and cnt[i + 1][j] == k + 1: 
                if i + j + 1 > sum(fins[-1]):
                    fins = [(i + 1, j)]
                elif i + j + 1 == sum(fins[-1]): 
                    fins.append((i + 1, j))
            if j + 1 < n and cnt[i][j + 1] == k + 1: 
                if i + j + 1 > sum(fins[-1]):
                    fins = [(i, j + 1)]
                elif i + j + 1 == sum(fins[-1]): 
                    fins.append((i, j + 1))
if cnt[-1][-1] <= k: 
    print('a' * (2 * n - 1))
    exit()
ans = []
while len(fins): 
    nfins = set() 
    fin = min(a[i][j] for i, j in fins)
    for i, j in fins: 
        if a[i][j] == fin: 
            if i + 1 < n: nfins.add((i + 1, j))        
            if j + 1 < n: nfins.add((i, j + 1))
    ans.append(fin)
    fins = nfins

print('a' * (2 * n - 1 - len(ans)) + ''.join(ans))

# 9.11
## T1. [Processing Queries](https://codeforces.com/problemset/problem/644/B)
- 解题思路: 模拟，计算每个任务的最早完成时间，当前的查询到达时，还有k个任务没有完成，则当前查询不会执行

In [ ]:
# url: https://codeforces.com/contest/644/submission/280674773
n, m = map(int, input().split())
a = [0] 
i = 0
outs = []
for _ in range(n):
    t, d = map(int, input().split()) 
    while i < len(a) and a[i] <= t: 
        i += 1 
    outs.append("-1" if len(a) - i > m else f"{max(a[-1], t) + d}")
    if outs[-1] != "-1": 
        a.append(int(outs[-1]))
print(*outs)

## T2. [Sereja and Periods](https://codeforces.com/problemset/problem/314/B)
- 解题思路
  1. 首先去掉无用变量，发现d是没有用的，只需要计算w中可以包含多少个c，然后除以d即可得到最终答案
  2. 考虑如何计算w中最多可以包含多少个c，使用贪心的寻找，假设当前匹配到了c的第i个字符，为了让w中包含尽量多的c，下一个匹配的字符一定是最接近的字符，因此只需要计算下一个需要匹配的位置，以及中途匹配成功的次数即可

In [ ]:
# https://codeforces.com/contest/314/submission/280677031
b, d = map(int, input().split()) 
a, c = input(), input() 

m = len(c)

next_idx = list(range(m))
cnt = [0] * m
for i in range(m): 
    for ch in a: 
        if ch == c[next_idx[i]]: 
            next_idx[i] += 1 
            if next_idx[i] == m: 
                cnt[i] += 1 
                next_idx[i] = 0 
ans = idx = 0 
for _ in range(b):
    ans += cnt[idx]
    idx = next_idx[idx]
print(ans // d)

# 9.12
## T1. [Elections](https://codeforces.com/problemset/problem/1267/E)
- 题解: 考虑到$n \le 100$，范围较小，枚举每个候选人，每次将该候选人与反对派候选人选票差距最大的那个票仓去掉即可

In [ ]:
# submission url: https://codeforces.com/contest/1267/submission/280795168

n, m = map(int, input().split()) 
a = [list(map(int, input().split())) for _ in range(m)]

ans = list(range(m))

for i in range(n - 1):
    b = [a[j][i] - a[j][-1] for j in range(m)]
    s = sum(b)
    if s >= 0: 
        print(0)
        exit() 
    c = sorted(range(m), key=lambda x: -b[x])
    res = []
    while s < 0: 
        s -= b[c[-1]]
        res.append(c[-1])
        c.pop() 
    if len(res) < len(ans): 
        ans = res 

print(len(ans))
print(' '.join(str(x + 1) for x in ans))

## T2. [TV Shows](https://codeforces.com/problemset/problem/1061/D)
- 题解
  1. 首先考虑当前已经选了电视了，是否还需要在选，可以发现，如果该电视上一个节目到当前节目的空闲租赁费比重新租一个电视的低，则选已有的
  2. 于是，可以使用堆或者任意排序的数据结构，存上一个电视的最后一个节目的结束时间，每次找打离当前电视开始时间最近的电视，然后判断1即可

In [ ]:
# submission url: https://codeforces.com/contest/1061/submission/280796720

n, x, y = map(int, input().split()) 
a = [list(map(int, input().split())) for _ in range(n)]
 
a.sort() 
b = SortedList()
ans = 0
for l, r in a: 
    ans += (r - l) * y
    ll = bisect.bisect_right(b, - l)
    if len(b) == ll or (l + b[ll]) * y > x: 
        ans += x 
    else:
        ans += (l + b[ll]) * y
        b.remove(b[ll])
    b.add( - r)  
        
print(ans % MOD)

# 9.13
## T1. [Tennis Championship](https://codeforces.com/problemset/problem/735/C)
- 题意: $n$个玩家参加比赛，若两个玩家参加比赛的数目相差超过1，则此二人不能比赛，求最多参加比赛的个数
- 题解: 最终比赛的进程会形成一棵二叉树，对于任意节点，其左右子树高度要么相等，要么差1，这样消耗的玩家个数是最小的，同时可以保证一个玩家进行的对局个数最多。

In [ ]:
# submission url: https://codeforces.com/contest/735/submission/281305096

n = int(input())
dp = [0] * 100
dp[1], dp[2] = 2, 3 
if n >= 2:
    ans = 1 
if n >= 3:
    ans = 2
for i in range(3, 100):
    dp[i] = dp[i - 1] + dp[i - 2]
    if dp[i] <= n: 
        ans = i
    else:
        break  
print(ans)

# 9.14
## T1. [Alyona and the Tree](https://codeforces.com/problemset/problem/682/C)
- 题意: 给定一棵树，每个节点和边都有权值，去掉所有从根节点到该节点的权值之和大于该节点权值的节点
- 题解: 逆向思考，这道题可以转换成找节点$v$，从该节点到所有祖先节点的权值之和最大值大于$\mathbf w_v$，去掉以该节点为根的所有子树，找最大节点可以用`dp`解决
- **小技巧**: 将递归转换成递推，模拟栈操作即可

In [ ]:
# submission url: https://codeforces.com/contest/682/submission/281062902

n = int(input())
a = list(map(int, input().split())) 

edge = [0]
tree = [[] for _ in range(n)]
for u in range(1, n):
    p, c = map(int, input().split())
    edge.append(c)
    tree[p - 1].append(u)

stk = [0] 
dp = [0] * n 
ans = 0 

while stk: 
    u = stk.pop()
    ans += 1 
    for v in tree[u]: 
        dp[v] = max(0, dp[u]) + edge[v]
        if dp[v] <= a[v]: 
            stk.append(v)

print(n - ans)

## T2. [Side Transmutations](https://codeforces.com/problemset/problem/1065/E)
- 题意: 给定三个数$n, m, |A|$分别代表字符串长度，下标数组$b$长度，候选字母个数，找出互不相同的所有字符串个数。其中，如果可以从$b$中选择下标i，让前$b_i$个字符构成的子串和后$b_i$个字符构成的子串反转后互换位置可以形成另一个字符串，则两字符串相同。
- 题解: 操作可以看成是对每个区间的子字符串独立进行的，因为总是可以转换成对某段独立操作，例如，对$b_1\to b_2$的中间子串进行操作，等价于先操作$b_2$再操作$b_1$。于是考虑对某个长度为$k$的段进行操作，候选的字符串集合为$\mathcal A = \{\mathcal A_1, \cdots, \mathcal A_{|A|^k}\}$。我们需要从中选择两个字符串，满足$0 \le i \le j \le |A|^k$即可，因为如果同时选择$(\mathcal A_i, \mathcal A_j)$和$(\mathcal A_j, \mathcal A_i)$，这两个字符串就可以通过固定次数操作得到，于是可以选择的方案个数为$C_{|A|^k + 1}^2$，对于大于$b_m$的部分，任意选择字符皆可，于是最终答案为$\sum_{i = 1}^n C_{|A|^{b_i - b_{i - 1}} + 1}^2 + |A|^{n - 2 * b_m}$.

In [ ]:
# submission url: https://codeforces.com/contest/1065/submission/281077341

MOD = 998244353
n, m, s = map(int, input().split())

ans, cur = 1, 0

for num in map(int, input().split()):
    p = pow(s, num - cur, MOD)
    ans *= p * (p + 1) // 2 % MOD 
    ans %= MOD 
    cur = num 

print(ans * pow(s, n - cur * 2, MOD) % MOD)

# 9.16
## T1. [Little Artem and Dance](https://codeforces.com/problemset/problem/641/C)
- 题意: $n$个人，初始编号为$1 \to n$，给定两类操作: 操作1，循环左移或者右移；操作2，与相邻下标互换，给定$q$次查询，问最后的位置情况
- 题解:
  一道比较典的题，考虑下标数组$p$的变化情况，对于两次操作可以分别做如下转换:  
  ____
  Operation 1. 位移量为$x$, 操作1等价于$p_i + x$ 

  Operation 2. 对于变化后的下标数组$p\prime$，操作2等价于，若$p\prime_i + \delta$，其中当$p\prime_i$为偶数，$\delta = 1$；反之$\delta = -1$

In [ ]:
n, q = map(int, input().split())
x = cnt = 0

for _ in range(q): 
    op = list(map(int, input().split()))
    if op[0] == 2: 
        if ((x + cnt) % n) % 2: 
            cnt -= 1 
        else:
            cnt += 1
    else:
        x += op[1] 
ans = [-1] * n 
for i in range(n):
    pos = (x + cnt + i) % n
    ans[pos] = i + 1
    cnt *= -1 
print(' '.join(str(x) for x in ans))

## T2. [Dwarves, Hats and Extrasensory Abilities](https://codeforces.com/problemset/problem/1063/C)
- 题意: 给出$n$个点的坐标，要求在这些点随机分为两类的前提下，能够通过一条直线分成两部分
- 题解: 考虑二维的场景比较困难，**可以先在一维场景下进行考虑**，化繁为简。考虑在一条直线上选点，可以选择点的区间为$[l, r]$，假定$l$为`black`，$r$为`white`，此时选择一个点$l\le x \le r$，若该点为`black`，由于不确定下一个点是什么类型，此时应该尽量让新的点加到$[x, r]$之间，换言之，两类点应该**尽量靠近线段两端**，此时可以贪心的选择$x = \frac{l + r}{2}$。考虑二维情形，按照上述方式选择，最终得可选区间可能为$[l, l + 1]$，即其中无法确定可分线段，此时考虑分割线段为$(l, r), (r, l)$即可。

In [ ]:
def query(x, y):
    print(x, y, flush=True)
    color = input()
    return color == 'black'

n = int(input())
l, r = 0, 10 ** 9 
init = query(r, r)
for _ in range(n - 1):
    x = l + r >> 1   
    if query(x, x) == init: 
        r = x
    else:
        l = x
print(l, r, r, l)

# 9.17
## T1. [Ryouko's Memory Note](https://codeforces.com/problemset/problem/433/C)
### 题意
给定一个长度为`n`的数组`a`，选择其中的一个数，并将`a`中所有相同的数变成另一个数，求$\sum_{i = 1}^{m - 1} |a_{i + 1} - a_i|$的最小值
### 题解
可以发现最终答案只与某个数两端的数有关，在修改某一个数时，最多也只会影响到周围的两个数，因此，对`a`中所有不同的数独立考虑，对于某个数$t$，获取相邻位置的元素，得到数组`b`，最终可以转化为求$\sum_{i = 1}^{|b| - 1} |b_i - t|$的最小值，这就转换成了经典的中位数贪心，即将`b`排序后，$t=b_{\left\lfloor \frac{|b|}{2}\right\rfloor}$时，达到最小。只需要枚举所有不同的数，独立进行上述计算，即可得到最终答案的最小值。

In [ ]:
# submission url: https://codeforces.com/contest/433/submission/281553330

n, m = map(int, input().split()) 
a = list(map(int, input().split())) 

vs = defaultdict(list)

ans = 0

for i, x in enumerate(a):
    vs[x].append(i)
    if i: 
        ans += abs(a[i] - a[i - 1])

ans_ = ans

for v, idxs in vs.items():
    q = []
    res = res_ = 0 
    for i in idxs: 
        if i and a[i] != a[i - 1]: 
            q.append(a[i - 1])
        if i < m - 1 and a[i] != a[i + 1]: 
            q.append(a[i + 1])
    if len(q) == 0: 
        continue
    q.sort()
    tgt = q[len(q) // 2]
    for x in q: 
        res_ += abs(v - x)
        res += abs(tgt - x)
    
    ans = min(ans, ans_ - res_ + res)
print(ans)

## T2. [Subordinates](https://codeforces.com/problemset/problem/729/E)
### 题意
给定$n$个工人，每个工人自述的领导个数为$a_i$，其中第$s$个人领导，其无领导，除此之外每个工人只有一个领导，给定`a`数组，问最少有多少个人的说法是错误的
### 题解
- tips1. 一个工人只有一个领导，但是一个领导不止领导一个工人
- tips2. 若$a_i$正确，则$a_i - 1$必须要存在于`a`中

由上述tips，对于`a`数组，修改最少的情况就是找一个最小的最大值$m$，让修改其中元素，使`a`满足$a_i \in \{0, 1, \cdots, m\}$，即让$\text {mex}\ a = \max\ a + 1$。代码实现中，每次访问找到当前的`mex`值，贪心地将最大值`max`变成`mex`，直至最终满足条件。

In [ ]:
# submission url: https://codeforces.com/contest/729/submission/281559910

n, s = map(int, input().split()) 
a = list(map(int, input().split()))
cnt = [0] * (n + 2)

for x in a: 
    cnt[x] += 1 

if a[s - 1]:
    ans = 1 
    cnt[-1] += cnt[0]
    cnt[a[s - 1]] -= 1 
else:
    ans = 0 
    cnt[-1] += cnt[0] - 1

mx  = n + 1
mex = 1 
while mex < mx:
    while mex < mx and cnt[mx] == 0: 
        mx -= 1
    while mex < mx and cnt[mex]: 
        mex += 1
    if mex < mx: 
        ans += 1 
        cnt[mx]  -= 1
    mex += 1 
print(ans + cnt[-1])

# 9.18
## T1. [Santa's Bot](https://codeforces.com/problemset/problem/1279/D)
### 题意
- 一道简单的概率题
给定一个长度为$n$的数组$a$，每个元素是一个数组$k$，包含$|k|$个数，每个元素都是等概率被选择，给定选择序列，求选择的结果有效的概率
### 题解
选到第$a_i$个数组的概率是$\frac{1}{n}$, 从第$a_i$个数组中选到数$j$的概率是$\frac{1}{|a_i|}$，则选到$j$的总概率为$\sum_{i = 1}^{n}\mathbb 1(j\in a_i) \frac{1}{n\cdot |a_i|}$. 最后将$j$分配给包含该数的数组概率是$\frac{\sum_{i = 1}^n \mathbb 1(j \in a_i)}{n}$. 故最终答案为$\frac{\sum_{i = 1}^n \mathbb 1(j \in a_i)}{n} \cdot \sum_{i = 1}^{n}\mathbb 1(j\in a_i) \frac{1}{n\cdot |a_i|}$.

In [ ]:
# submission url: https://codeforces.com/contest/1279/submission/281694198
mod = 998244353

n = int(input())

cnt = [0] * (10 ** 6 + 1)
fin = [0] * (10 ** 6 + 1)

for _ in range(n):
    a = list(map(int, input().split()))
    for i in a[1:]:
        cnt[i] += 1 
        fin[i] += pow(n * a[0], -1, mod)
        fin[i] %= mod 
ans = 0 
for x, y in zip(cnt, fin):
    ans += y * x * pow(n, -1, mod) % mod 
    ans %= mod 

print(ans)

## T2. [Iahub and Permutations](https://codeforces.com/problemset/problem/340/E)
### 题意
给定一个长度为$n$的数组$a$，其中值为`-1`的位置可以自由变化数值，满足$a_i \ne i$，求可能的个数
### 题解
值为`-1`的位置集$\mathcal S$有两种情况: 
1. $i$已经在其它固定的位置出现，即$i \in \overline {\mathcal S}$，此时在该位置可以填入任意值
2. $i$还没有在其它位置出现，即$i \in \mathcal S$, 此时需要判断条件

设状态1的位置个数为$c_1$，状态2的位置个数$c_2$，考虑使用`dp`。设`dp[i]`表示$c_2$个元素中，已有$i$个元素不在其位置上的方案数，考虑第$i$个元素:
- 若其填入$\mathcal S_1$中的元素，则此时有$c_2\cdot dp_{i - 1}$种方案
- 考虑前$i - 1$个位置
  - 若该位置填入$i$，则此时至少有两个位置已经不与下标相等，故有$(i - 1) \cdot dp_{i - 2}$种方案
  - 若该位置没有填入$i$，则仍然只有一个位置已经满足条件，此时有$(i - 1) \cdot dp_{i - 1}$种方案
上述方案求和即为最终答案

In [ ]:
# submission url: https://codeforces.com/contest/340/submission/281700621
mod = 10 ** 9 + 7

n = int(input()) 
a = list(map(int, input().split())) 

cnt1 = cnt2 = 0 
for i, x in enumerate(a, 1):
    if x == -1: 
        if i in a: 
            cnt2 += 1 
        else:
            cnt1 += 1 

dp = [0] * (cnt1 + 1) # cnt1个元素中，恰好有i个元素不在本身位置上的方案个数
dp[0] = 1
for i in range(1, cnt2 + 1):
    dp[0] = (dp[0] * i) % mod

for i in range(1, cnt1 + 1): 
    dp[i] = cnt2 * dp[i - 1] + (i - 1) * dp[i - 1] + (i - 1) * dp[i - 2]
    dp[i] %= mod 
print(dp[-1])

# 9.19
## T1. [Nearest Fraction](https://codeforces.com/problemset/problem/281/B)
### 题意
给定三个数$x, y, n$，找$|\frac{x}{y} - \frac{a}{b}|$的最小值
### 题解
$n\le 10^5$，直接暴力做即可，做的时候注意精度问题

In [ ]:
# submission url: https://codeforces.com/contest/281/submission/281826103

x, y, n = map(int, input().split())

fin = [10 ** 15, 1]

for b in range(1, n + 1):
    for a in [x * b // y, (x * b + y - 1) // y]: 
        if abs(fin[1] * x - fin[0] * y) * y * b > abs(x * b - a * y) * fin[1] * y:
            fin = [a, b]
print(f"{fin[0]}/{fin[1]}")

# 9.23
## T1. [Vasya and Robot](https://codeforces.com/problemset/problem/1073/C)
### 题意
给你一个坐标$x, y$和一个操作序列，每次操作控制上下左右移动一格，初始为原点，求让中点为给定点的最小更改长度
### 题解
每次只能移动一格，那么我们有一个贪心的构造方式，让前n步直接到达终点，然后来回移动即可，因此，判断是否可达等价于$|x| + |y| \le n$，同时，移动一次会让$x + y$的奇偶性改变，因此$x + y$ 需要与 $n$奇偶性相同。

In [ ]:
class Solution:
    """
    problem   : https://codeforces.com/problemset/problem/1073/C
    submission: https://codeforces.com/contest/1073/submission/282577034
    """
    @staticmethod
    def main(ac=FastIO()):
        n = ac.read_int()
        s = ac.read_str()
        x, y = ac.read_list_ints()
        
        if abs(x) + abs(y) > n or (x + y - n) % 2: 
            ac.st(-1)
            return 
        dirs = {
            'U': (0, 1),
            'D': (0, -1), 
            'L': (-1, 0), 
            'R': (1, 0)
        }
        acc_x, acc_y = [0] * (n + 1), [0] * (n + 1)  
        for i, op in enumerate(s): 
            dx, dy = dirs[op] 
            acc_x[i + 1] = acc_x[i] + dx 
            acc_y[i + 1] = acc_y[i] + dy
        
        def ok(l, r):
            return abs(acc_x[l] + acc_x[-1] - acc_x[r] - x) + abs(acc_y[l] + acc_y[-1] - acc_y[r] - y) <= r - l
        
        ans = n
        l = r = 0 
        while l <= n: 
            while r < n and not ok(l, r): 
                r += 1 
            if not ok(l, r): break
            ans = ac.min(ans, r - l)
            l += 1 
        ac.st(ans)

## T2. [Tyler and Strings](https://codeforces.com/problemset/problem/1648/C)
### 题意
给定两个字符串$s$和$t$，求$s$的子串中重新排列后字典序比$t$小的个数
### 题解
考虑从$0\sim j$的字母已经确定，要让$s\prim$比$t$小，只需要让$s_{j + 1} < t_{j + 1}$，则后面的字母可以随便排；同时若$s_{j + 1} = t_{j + 1}$，则继续看后面的位置，此时可能的组合数为$(n - j - 1)!$；然而重复数字不可重复计算，对于$k$，重复计算了$k!$次，除去即可，最终答案需要加上$cnt\frac{(n - j - 1)!}{\prod_i^mx c_i!}$。
### Tips
考虑后半部分，发现当$j$变成$j + 1$时，相当于分母除以$n - j - 1$，于是维护答案时，可以用逆元；考虑前半部分，$cnt$为小于当前值$v$的字母个数，可以考虑使用树状数组计算。

In [ ]:
class Solution:
    """
    problem   : https://codeforces.com/problemset/problem/1648/C
    submission: https://codeforces.com/contest/1648/submission/282622576
    """
    @staticmethod
    def main(ac=FastIO()):
        n, m = ac.read_ints()
        s = ac.read_ints()
        t = ac.read_ints()

        M = 2 * 10 ** 5 + 1 
        cnt = [0] * M  # 记录s中每个数出现个数
        fen = FenwickTree(M)  # 树状数组存的是s中每个小于k个数个数

        for x in s:
            cnt[x] += 1 
            fen.add(x, 1)

        fac = Factorial(n, ac.MOD2)
        # 初始化
        cur = fac.fac(n)
        for v in cnt:
            cur *= fac.fac_inv(v)
            cur %= ac.MOD2
        ans = 0 
        for i, v in enumerate(t):
            if i >= n: 
                # s只能为t的前缀
                ans += 1 
                break 
            cur = cur * fac.inv(n - i) % ac.MOD2  # 除以n-i
            ans += (cur * fen.rsum(0, v - 1)) % ac.MOD2
            ans %= ac.MOD2

            if cnt[v] == 0: break  # 没有相等的，无法继续计算
            cur = cur * cnt[v] % ac.MOD2
            cnt[v] -= 1 
            fen.add(v, -1)
        ac.st(ans % ac.MOD2)

# 9.24
## T1. [Dungeons and Candies](https://codeforces.com/problemset/problem/436/C)
### 题意
给定$k$个$n \times m$的字母矩阵, 每个矩阵要么变成空, 这需要消耗$n \times m$; 要么与另一个矩阵合并，这需要消耗$D_{A, B}\times w$, 求把每个矩阵消除的最小消耗, 其中$D$为两个矩阵的不同字母个数
### 题解
显然, 消除的路线最终会构成一个森林, 初始时可以看成一个完全图, 每个节点是矩阵, 边是矩阵消除的代价, 等价于寻找图的最小生成树。可以使用*prim*算法或者*cuscal*算法。建图的时候需要注意, 构建一个虚节点, 表示直接将矩阵消除

In [ ]:
"""
problem url   : https://codeforces.com/problemset/problem/436/C
submission url: https://codeforces.com/contest/436/submission/282735412
"""


n, m, k, w = MII()
d = [0] * (k + 1) ** 2
a = [[I() for _ in range(n)] for _ in range(k)]

def f(i, j):
    return i * (k + 1) + j 

for i in range(k):
    d[f(i, k)] = n * m 
    for j in range(i):
        cnt = 0 
        for l in range(n):
            for r in range(m):
                cnt += a[i][l][r] != a[j][l][r]
        d[f(i, j)] = d[f(j, i)] = cnt * w
fin = 0 
dis = [k] * (k + 1)
st  = set(range(k + 1))
while st: 
    cur = -1 
    for x in st: 
        if cur == -1 or d[f(x, dis[x])] < d[f(cur, dis[cur])]: 
            cur = x 
    fin += d[f(cur, dis[cur])] if cur != k else 0 
    st.remove(cur)
    for v in st: 
        if d[f(v, dis[v])] > d[f(v, cur)]:
            dis[v] = cur  

outs = [] 
stk = [k]
while stk: 
    u = stk.pop() 
    for v in range(k):
        if dis[v] == u: 
            outs.append(f"{v + 1} {u + 1 if u != k else 0}")
            stk.append(v)
                
print(fin)
print('\n'.join(outs))

# 9.25
## T1. [Marcin and Training Camp](https://codeforces.com/problemset/problem/1210/B)
### Formulation
给定$n$个人，每个人包含一个$a_i, b_i$，分别表示会做哪些题，和选择他会带来多大价值，要求选择的人中不能有一个人会其他所有人都不会的题，计算最后的最大价值之和
### Solution
选择$x$时，最终数组中一定要有一个人$y$，要么$a_x = a_y$，要么$a_x$是$a_y$的子集；以此类推，最后一定有两个人会的题相同，因此枚举最终状态，然后依次寻找满足条件的人即可。


In [ ]:
"""
problem url   : https://codeforces.com/problemset/problem/1210/B
submission url: https://codeforces.com/contest/1210/submission/282853166
"""

n = II() 
a = LII() 
b = LII() 

cnt = defaultdict(int)
for x in a: 
    cnt[x] += 1 
used = [0] * n 
ans = 0

for x, c in cnt.items():
    if c < 2: continue
    for i, [y, v] in enumerate(zip(a, b)):
        if not used[i] and x & y == y:  # 子集
            ans += v 
            used[i] = 1

print(ans)

## T2. [Dasha and Puzzle](https://codeforces.com/problemset/problem/761/E)
### 题意
给定$n$个点，在平面上构造一棵树，要求每个节点周围的邻居不得多于4个
### 题解
- 显然，如果一个节点的邻居个数>=4，最终一定不可能构建这棵树
- 于是解这道题的关键在于如何让两条边不相交
以边的长度为主构建树，具体来说，让第一条边足够长，后续构建的所有的边都去这个长度的一部分即可；同时取与上一条边的不同方向，即可达成最终结果。

In [ ]:
"""
problem url   : https://codeforces.com/problemset/problem/761/E
submission url: https://codeforces.com/contest/761/submission/282874126
"""

n = II()
adj = [[] for _ in range(n)]
for i in range(n - 1):
    u, v = GMI() 
    adj[u].append(v)
    adj[v].append(u)

dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]

ans = [None] * n 
ans[0] = (0, 0)
stk = [(0, -1, 1 << 30)]
while stk: 
    u, dr, length = stk.pop()
    cur = 0
    if len(adj[u]) > 4: 
        print("NO")
        exit()
    for v in adj[u]: 
        if ans[v] is None:
            if cur == dr: 
                cur += 1 
            dx, dy = dirs[cur % 4]
            x, y = ans[u][0] + dx * length, ans[u][1] + dy * length
            ans[v] = (x, y)
            stk.append((v, cur ^ 2, length // 2))
            cur = (cur + 1) % 4
print("YES")
print('\n'.join([f"{i} {j}" for i, j in ans]))

# 9.26
## T1. [XK Segments](https://codeforces.com/problemset/problem/895/B)
### 题意
给定$n$个数，求满足$a_i \le y \le a_j$且$y$能被$x$整除，这样的$y$的个数为$k$的下标对$(i, j)$的个数
### 题解
首先这道题并不关心下标的顺序，因此可以排序，对于有序的序列，对于下标对$(i, j), i \le j$，判断条件等价于$ \left \lfloor \frac{a_j}{x} \right \rfloor - \left \lfloor \frac{a_i - 1}{x}  \right \rfloor = k$，因此用字典记录$\left \lfloor \frac{a_i - 1}{x}  \right \rfloor$的个数，每次查找$\left \lfloor \frac{a_j}{x} \right \rfloor - k$的个数；同时，如果两个数相等，则需要考虑下标逆向的时候，因此同时记录$a_j$的个数。

In [ ]:
"""
problem url   : https://codeforces.com/problemset/problem/895/B
submission url: https://codeforces.com/contest/895/submission/282990684
"""
n, x, k = MII() 
a = LII() 
a.sort()
cnt, cntv = Counter(), Counter()
ans = 0 

for y in a:
    cnt[(y - 1) // x] += 1 
    ans += cnt[y // x - k]
    if y // x - (y - 1) // x == k: 
        ans += cntv[y]
    cntv[y] += 1 

print(ans)

## T2. [Ghosts](https://codeforces.com/problemset/problem/975/D)
### 题意
给定$n$个坐标，初始均在一条直线上，每个点有移动速度$vi = v_{ix} + v_{iy}$，求交点个数
### 题解
简单的数学推导题。相交的充要条件: $v_{iy} - v_{ix} \times a = v_{jy} - v_{jx} \times a$，其中$v_{ix} \neq v_{ijx}$， 且$v_{iy} \neq v_{jy}$。

In [ ]:
"""
problem url   : https://codeforces.com/problemset/problem/975/D
submission url: https://codeforces.com/contest/975/submission/283000631
"""

n, a, b = MII() 
ans = 0 
cnt = Counter()
cntv = Counter()

for _ in range(n):
    x, vx, vy = MII() 
    if vy - vx * a in cnt:
        ans += cnt[vy - vx * a] - cntv[(vx, vy)]
    cnt[vy - vx * a] += 1
    cntv[(vx, vy)] += 1 

print(ans * 2) 